In [67]:
import psycopg2
import json

# Replace with your actual database credentials
dbname = "initial_db"
user = "postgres"
password = "P6ppdnhCmw4nG8kU2a1K"
host = "tunein-db.che0om0o6j54.af-south-1.rds.amazonaws.com"
port = 5432  # Note: Port should be an integer, not a string

In [68]:
conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )

In [69]:
# import json file called output.json and convert it to a dictionary
with open('combined_playlists.json') as f:
    data = json.load(f)

In [70]:
# print(data["playlists"]["playlist_1"][0]["id"])
print(len(data["playlists"]))
dataObject = {}
dataObject["playlists"] = "playlist_1"
print(dataObject)

20
{'playlists': 'playlist_1'}


In [71]:
cursor = conn.cursor()

In [72]:
cursor.execute("select room_id from room;")
rooms = cursor.fetchall()
# extract room ids from the list of tuples
room_ids = [room[0] for room in rooms]
print(room_ids)

['2df818b9-876e-463a-b3cd-ed78e62b0966', '5f9e828e-8957-4554-9a98-cefc880c42ee', '12cee178-cb1f-484a-a062-cd3510e0feab', '9aaff1d1-c1f2-4422-80e5-71a66d5225f2', '66bb6bf7-25be-45af-bc38-7e7e258797b8', '900dd6f1-440a-4272-a215-cf00ef78a9cf', '1318c800-3fb9-47be-b901-443a6582aacd', '57944007-200e-4fa9-bcc1-fedcde15f435', 'e71ddd8a-67d2-40e6-a5c8-539d51e0ca8a', 'b9b4aebd-36cd-40bb-b329-f394fdefa0f5', '3abae5ed-7de7-4f4f-833c-07de8a117479', 'cedf3962-a82a-460b-9409-f82161acd93d', 'ca54e19b-7385-485e-b16c-f89bfe132b3c', '7725f3b6-9d14-4c2a-84d7-84984204251e', '8f928675-5c95-497a-b8a7-917064cdb462', '7073a1d7-555b-4a6e-843e-d0f048983ad5', '18ee1515-a910-4bf0-8b26-751cb2bad811', 'a7d3cad5-46d2-40e8-b623-f2fd83d3a1e6', 'cc0a74c5-e2b0-43c2-b0f7-76587a0a003c', '777b6f7c-71f3-4ad5-849b-5eea361d7d87']


In [73]:
for playlist in data["playlists"]:
    try: 
        room = room_ids.pop(0)
    except IndexError:
        print("No more rooms")
        break
    for song in data["playlists"][playlist]:
        # print(song)
        audio_features = {}
        audio_features["danceability"] = song["danceability"]
        audio_features["energy"] = song["energy"]
        audio_features["key"] = song["key"]
        audio_features["loudness"] = song["loudness"]
        audio_features["mode"] = song["mode"]
        audio_features["speechiness"] = song["speechiness"]
        audio_features["acousticness"] = song["acousticness"]
        audio_features["instrumentalness"] = song["instrumentalness"]
        audio_features["liveness"] = song["liveness"]
        audio_features["valence"] = song["valence"]
        audio_features["tempo"] = song["tempo"]
        # print(type(eval(song["artists"])))

        # print(audio_features)
        # insert into the song table the values name, artist (as a string), spotify_id, duration, artwork_url and audio_features (as a json object)
        cursor.execute(
            "INSERT INTO song (name, artists, spotify_id, duration, audio_features) VALUES (%s, %s, %s, %s, %s) returning *",
            (
                song["name"], 
                eval(song["artists"]), 
                song["id"], 
                song["duration_ms"]/1000,
                json.dumps(audio_features)
            )
        )
        result = cursor.fetchone()
        print(result)
        cursor.execute(
            "INSERT INTO queue (room_id, song_id) VALUES (%s, %s) returning *",
            (room, result[0])
        )
        queue_result = cursor.fetchone()
        print("queue result:", queue_result)
        cursor.connection.commit()

('287c3343-135e-4128-8646-3470c9c5f258', 'The Birds', None, None, 288, None, None, '2sraohMaCPdByBdDcsaB62', {'danceability': 0.593, 'energy': 0.74, 'key': 4, 'loudness': -6.04, 'mode': 1, 'speechiness': 0.0389, 'acousticness': 0.185, 'instrumentalness': 0.86, 'liveness': 0.126, 'valence': 0.488, 'tempo': 151.273}, ['Rick Cuevas'])
queue result: ('2df818b9-876e-463a-b3cd-ed78e62b0966', '287c3343-135e-4128-8646-3470c9c5f258', False, '21fdc789-d185-4af1-87f5-6c17a0b548e0', None, datetime.datetime(2024, 8, 27, 17, 45, 18, 251387, tzinfo=datetime.timezone.utc))


('857b1f87-a761-4e4a-8deb-626416991569', 'Spellbound (1945) Spellbound Concerto', None, None, 731, None, None, '2wke5oNlPUAc3wAcxXXn7Y', {'danceability': 0.283, 'energy': 0.239, 'key': 8, 'loudness': -13.962, 'mode': 1, 'speechiness': 0.0289, 'acousticness': 0.884, 'instrumentalness': 0.614, 'liveness': 0.108, 'valence': 0.0506, 'tempo': 85.431}, ['Miklós Rózsa', 'Hollywood Bowl Orchestra'])
queue result: ('2df818b9-876e-463a-b3cd-ed78e62b0966', '857b1f87-a761-4e4a-8deb-626416991569', False, 'fadcbe3a-d5e9-465a-b5f4-5cbbac1f13bd', None, datetime.datetime(2024, 8, 27, 17, 45, 18, 437988, tzinfo=datetime.timezone.utc))
('c58a14da-342d-4530-bcc3-0fcdc9646f6f', 'Demons (feat. Fivio Foreign & Sosa Geek)', None, None, 205, None, None, '05aZ9sAU1YXndHv0FMi9iW', {'danceability': 0.5429999999999999, 'energy': 0.7659999999999999, 'key': 0, 'loudness': -3.387, 'mode': 0, 'speechiness': 0.343, 'acousticness': 0.423, 'instrumentalness': 0.0, 'liveness': 0.155, 'valence': 0.7290000000000001, 'tempo'

In [74]:
cursor.execute("insert into genre (genre) values ('super duper hip hop') returning *;")
# get newly created row
rows = cursor.fetchone()
print(rows)

('6cc2745b-3e6d-4f4f-9e41-7d5ac30d5940', 'super duper hip hop')


In [75]:
cursor.execute("SELECT * from users;")
rows = cursor.fetchall()
print(type(rows[0][0]))


    # Print each row
for row in rows:
    # Encode each column before printing
    encoded_row = [str(col).encode('utf-8').decode('unicode_escape') if isinstance(col, str) else col for col in row]
    print(encoded_row)

<class 'str'>
['01ece2d8-e091-7023-c1f2-d3399faa7071', 'farmer 345', 'Music enthusiast who loves exploring new tunes across various genres. Always on the lookout for fresh beats and hidden gems!', 'https://tunein-nest-bucket.s3.af-south-1.amazonaws.com/IMG_20230913_153948_541.jpg', {'recent_rooms': ['0352e8b8-e987-4dc9-a379-dc68b541e24f', '497d8138-13d2-49c9-808d-287b447448e8', '376578dd-9ef6-41cb-a9f6-2ded47e22c84', '62560ae5-9236-490c-8c75-c234678dc346']}, {'fav_genres': ['Rock', 'Indie'], 'fav_songs': [{'title': 'Faster', 'artists': 'Good Kid', 'cover': 'https://store.goodkidofficial.com/cdn/shop/products/GoodKidAlbumCover.jpg?v=1528948601', 'start_time': ''}]}, 'Farmer23', {'data': [{'type': 'instagram', 'links': 'instagram.com/farmer'}, {'type': 'tiktok', 'links': 'tiktok.com/farmer'}]}, 'farmer345.jk@gmail.com']
['62d34a85-4ed0-4726-ab5f-aae5a90e4532', 'khensani', 'lets be friends', 'https://tunein-nest-bucket.s3.af-south-1.amazonaws.com/IMG_20230913_153948_541.jpg', None, {'fav_

In [76]:
cursor.close()